In [ ]:
!pip install datasets
!pip install transformers

In [9]:
from datasets import load_dataset


from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    # Seq2SeqTrainingArguments,
    # Seq2SeqTrainer,
    # DataCollatorForSeq2Seq,
)


In [ ]:
languaje = 'english'

model_name = "sshleifer/distilbart-xsum-12-3"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:

book_data = load_dataset("ubaada/booksum-complete-cleaned", "books")

In [ ]:
train = book_data['train']
test = book_data['test']
validation = book_data['validation']

In [ ]:
# def flatten(ds):
#     text = book_data['train']['text']
#     summary = book_data['train']['summary']
#     zipped = zip(text, summary)
#     return dict(zipped)

In [17]:

# tokenization
encoder_max_length = 256  # demo
decoder_max_length = 64

In [21]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["text"][0], batch["summary"][0]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

In [22]:
train_data = train.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train.column_names,
)


Map:   0%|          | 0/151 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [16]:
print(book_data['train'])
# print(book_data.shape[0])

Dataset({
    features: ['bid', 'title', 'text', 'summary'],
    num_rows: 151
})
